In [0]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/32/93/66826e2f50cefecbb0a44bd1e667316bf0a3c8e78cd1f0cdf52f5b2c5c6f/xgboost-2.1.3-py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for nvidia-nccl-cu12 from https://files.pythonhosted.org/packages/ed/1f/6482380ec8dcec4894e7503490fc536d846b0d59694acad9cf99f27d0e7d/nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB 1.0 MB/s eta 0:02:31
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/153.9 MB 2.0 MB/s eta 0:01:16
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/153.9 MB 3.8 MB/s eta 0:00:41
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.9/153.9 MB 6.6 MB/s eta 0:00:24
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/153.9 MB 11.9 MB/s eta 0:00:13
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/153.9 MB 21.4 MB/s eta 0:00:07
   ━╸━━

In [0]:
import joblib
trained_model=joblib.load('xgboost_model.pkl')

In [0]:
import mlflow
from mlflow.models import infer_signature
import numpy as np
from sklearn.model_selection import train_test_split


# Đọc dữ liệu và xử lý
path = '/mnt/gold/gold/'
df = spark.read.format('delta').load(path)

required_columns = [
    'is_injured', 'contributing_factor_vehicle_1_encoded', 'contributing_factor_vehicle_2_encoded',
    'contributing_factor_vehicle_3_encoded', 'contributing_factor_vehicle_4_encoded',
    'contributing_factor_vehicle_5_encoded', 'vehicle_type_code1_encoded', 
    'vehicle_type_code2_encoded', 'borough_encoded'
]
df_pd = df.select(required_columns).toPandas()

# Chia dữ liệu thành đặc trưng và nhãn
X = df_pd.drop(columns=['is_injured'])
y = df_pd['is_injured']

# Chia tập dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Bắt đầu ghi log trong MLflow
with mlflow.start_run(run_name="Prophet Model Run") as run:
    # Ghi signature của mô hình
    signature = infer_signature(X_train, y_train)

    # Đăng ký và ghi mô hình với MLflow
    reg_model_name = "xgboost_model"
    print("--")
    model_log=mlflow.sklearn.log_model(
        sk_model=trained_model,
        artifact_path="xgboost_model",
        serialization_format="cloudpickle",
        signature=signature,
        registered_model_name=reg_model_name,
    )

    # Lấy run_id để theo dõi sau này
    run_id = run.info.run_uuid
    print(f"Run ID: {run_id}")


/databricks/python/lib/python3.11/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


--


/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'xgboost_model'.
2024/12/01 07:32:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgboost_model, version 1


Run ID: 2118c84cc11c4900ab54451ff925ce80


Created version '1' of model 'xgboost_model'.


In [0]:
!pip install --upgrade --force-reinstall pyyaml


  Obtaining dependency information for pyyaml from https://files.pythonhosted.org/packages/75/e4/2c27590dfc9992f73aabbeb9241ae20220bd9452df27483b6e56d3975cc5/PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (762 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Not uninstalling pyyaml at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-3a0dca17-5172-433f-9fc4-c2e9d39c129e
    Can't uninstall 'PyYAML'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
 %restart_python 